This project has been made in Google Collab. So fork/download it in google collab adn the run it nicely.

# UPVOTE if you like this work  :)

# Face recognition with EfficientNetB2 :
 In this project we have taken a kaggle dataset to classify some human emotions.

The main motive of this project was to use the SOTA model produced by GOOGLE AI **EfficientNet** : *EfficientNetB2*.


The first work is we have docked the notebook with kaggle so that we can directly download the data straight into the collab notebooks directory. Then unzipped the folder and now we can use the data without downloading locally.

In [1]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
                                  
!cp kaggle.json ~/.kaggle/      #   (!cp [   API token file name   ] ~/.kaggle/)

!chmod 600 ~/.kaggle/kaggle.json

In [1]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

100%|███████████████████████████████████████▊| 120M/121M [00:22<00:00, 5.98MB/s]
100%|████████████████████████████████████████| 121M/121M [00:22<00:00, 5.65MB/s]


In [3]:
from zipfile import ZipFile
zf = ZipFile('face-expression-recognition-dataset.zip', 'r')
zf.extractall('./data')
zf.close()

As we know this neural network has a super long neuron layers , we have to use gpu for running it conveniently.
So we are installing tensorflow gpu to run it swiftly.

In [4]:
!pip install tensorflow-gpu

     |████████████████████████████████| 458.3 MB 4.9 kB/s eta 0:00:014
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 5.8 MB 10.6 MB/s eta 0:00:01
  Using cached wheel-0.37.0-py2.py3-none-any.whl (35 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached tensorflow_estimator-2.6.0-py2.py3-none-any.whl (462 kB)
     |████████████████████████████████| 14.8 MB 5.4 MB/s eta 0:00:01
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     |████████████████████████████████| 1.1 MB 6.0 MB/s eta 0:00:01
  Using cached clang-5.0.tar.gz (30 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached wrapt-1.1

Now checking what type of gpu we have been alotted.

In [1]:
!nvidia-smi

Tue Oct 26 20:01:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   70C    P0    59W /  N/A |   5420MiB /  5934MiB |     42%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Libraries:

In [1]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [2]:
from tensorflow.keras.layers import Lambda,Input,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB2



In [3]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [4]:
IMAGE_SIZE=[224,224]
train_path='/home/gauransh/Code/mirror-backend/notebooks/data/images/train'

valid_path='/home/gauransh/Code/mirror-backend/notebooks/data/images/validation'

Now adding the effnetb2.

In [5]:
eff=EfficientNetB2(input_shape=IMAGE_SIZE+[3],include_top=False)

2021-10-26 20:09:37.206737: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 20:09:37.215035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 20:09:37.215578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-26 20:09:37.216490: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

As we are wanting it to be pretrained we are stopping the layers from training on this data.

In [6]:
for layer in eff.layers:
  layer.trainable=False

In [7]:
folders=glob('/home/gauransh/Code/mirror-backend/notebooks/data/images/train/*')

In [8]:
folders

['/home/gauransh/Code/mirror-backend/notebooks/data/images/train/neutral',
 '/home/gauransh/Code/mirror-backend/notebooks/data/images/train/fear',
 '/home/gauransh/Code/mirror-backend/notebooks/data/images/train/angry',
 '/home/gauransh/Code/mirror-backend/notebooks/data/images/train/disgust',
 '/home/gauransh/Code/mirror-backend/notebooks/data/images/train/happy',
 '/home/gauransh/Code/mirror-backend/notebooks/data/images/train/sad',
 '/home/gauransh/Code/mirror-backend/notebooks/data/images/train/surprise']

In [9]:
x=Flatten()(eff.output)

We just want to change the last layer as we want to classify the emotions with desirable classes.

In [10]:
prediction=Dense(len(folders),activation='softmax')(x)

model=Model(inputs=eff.input,outputs=prediction)

Now checking the model.

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

We can see the model is huge. So we need a lot of time to run it.

In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Now making the datasets for training and testing.

In [13]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [14]:
training_set=train_datagen.flow_from_directory(train_path,
                                               target_size=(224,224),batch_size=32,class_mode='categorical')

Found 28821 images belonging to 7 classes.


In [15]:
print(train_path, valid_path)

/home/gauransh/Code/mirror-backend/notebooks/data/images/train /home/gauransh/Code/mirror-backend/notebooks/data/images/validation


In [16]:
test_set=test_datagen.flow_from_directory(valid_path,
                                               target_size=(224,224),batch_size=32,class_mode='categorical')

Found 7066 images belonging to 7 classes.


As running many epoch can take the whole day we are only showing 3 epochs to let you see how the model is getting tuned with the dataset.

In [17]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [18]:
import Image
print(Image.__file__)

/home/gauransh/Code/mirror-backend/lib/python3.7/site-packages/PIL/Image.py


In [19]:
r=model.fit_generator(training_set,validation_data=test_set,
                      epochs=10,
                      steps_per_epoch=len(training_set),
                      validation_steps=len(test_set))

/home/gauransh/Code/mirror-backend/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/gauransh/Code/mirror-backend/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
2021-10-26 20:09:51.391081: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2021-10-26 20:09:56.533628: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101


901/901 [==============================] - 199s 213ms/step - loss: 5.5332 - accuracy: 0.1751 - val_loss: 4.1500 - val_accuracy: 0.1446
Epoch 2/10
901/901 [==============================] - 213s 236ms/step - loss: 5.3651 - accuracy: 0.1769 - val_loss: 3.1460 - val_accuracy: 0.1359
Epoch 3/10
901/901 [==============================] - 197s 219ms/step - loss: 5.0200 - accuracy: 0.1750 - val_loss: 4.2710 - val_accuracy: 0.1441
Epoch 4/10
901/901 [==============================] - 200s 221ms/step - loss: 5.3290 - accuracy: 0.1716 - val_loss: 3.3004 - val_accuracy: 0.2583
Epoch 5/10
901/901 [==============================] - 194s 215ms/step - loss: 5.1608 - accuracy: 0.1720 - val_loss: 6.8384 - val_accuracy: 0.2583
Epoch 6/10
901/901 [==============================] - 210s 234ms/step - loss: 5.0466 - accuracy: 0.1725 - val_loss: 3.4456 - val_accuracy: 0.2583
Epoch 7/10
901/901 [==============================] - 209s 232ms/step - loss: 5.0520 - accuracy: 0.1720 - val_loss: 4.0378 - val_accura

In [20]:
model.save("saved_models/")

2021-10-26 21:46:06.949925: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_models/assets


/home/gauransh/Code/mirror-backend/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


This took us around 2 hours when connected with gpu.

and it seems that if you run it for 10 epochs you may get 92% accuracy or the validation set. So it certifies that the SOTA model works nicely.


If you love this project do leave a **star :)**



# Run this work on google collab. 
# Or you change the path of the dataset and run it in kaggle.